# Example Sample Hotel and Flight Booker Agent 

यो समाधानले तपाईंलाई फ्लाइट टिकटहरू र होटल बुक गर्न मद्दत गर्नेछ। परिदृश्य लन्डन हिथ्रो LHR बाट नयाँ योर्क JFK फेब्रुअरी २०, २०२४ मा यात्रा गर्न र फेब्रुअरी २७, २०२५ मा फर्कन बृटिश एयरवेजको इकनोमीमा मात्र उड्ने योजना हो। म नयाँ योर्कमा एउटा हिल्टन होटलमा बस्न चाहन्छु कृपया फ्लाइट र होटलको लागतहरू प्रदान गर्नुहोस्।


# Azure AI Agent Service सुरु गर्नुहोस् र **.env** बाट कन्फिगरेसन जानकारी प्राप्त गर्नुहोस्

### **.env**

एक .env फाइल सिर्जना गर्नुहोस्

**.env** मा Azure AI Agent Service को कनेक्शन स्ट्रिङ, AOAI द्वारा प्रयोग गरिएको मोडेल, र सम्बन्धित Google API Search सेवा API, ENDPOINT, आदि समावेश हुन्छ।

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "तपाईंको Azure AI Agent Service Model Deployment नाम"

[**NOTE**] तपाईलाई १००,००० दर सीमा (प्रति मिनेट टोकन) र ६०० (प्रति मिनेट अनुरोध) दर सीमा भएको मोडेल चाहिन्छ।

  तपाईं Microsoft Foundry - Model and Endpoint मा मोडेल प्राप्त गर्न सक्नुहुन्छ। 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "तपाईंको Azure AI Agent Service Project कनेक्शन स्ट्रिङ"

  तपाईं आफ्नो प्रोजेक्ट अवलोकनमा AI Foundry Portal स्क्रीनमा प्रोजेक्ट कनेक्शन स्ट्रिङ प्राप्त गर्न सक्नुहुन्छ।

- **SERPAPI_SEARCH_API_KEY** = "तपाईंको SERPAPI Search API कुञ्जी"
- **SERPAPI_SEARCH_ENDPOINT** = "तपाईंको SERPAPI Search Endpoint"

Azure AI Agent Service को Model Deployment Name र Project Connection String प्राप्त गर्न तपाईंले Azure AI Agent Service सिर्जना गर्नुपर्नेछ। यसलाई सिधा सिर्जना गर्न [यस टेम्प्लेट](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) प्रयोग गर्न सल्लाह दिइन्छ（***Note:*** Azure AI Agent Service हाल सीमित क्षेत्रमा सेट गरिएको छ। क्षेत्र सेट गर्न [यस लिंक](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) लाई सन्दर्भ गर्न सल्लाह दिइन्छ）

Agent लाई SERPAPI पहुँच आवश्यक छ। यसलाई [यस लिंक](https://serpapi.com/searches) प्रयोग गरेर दर्ता गर्न सल्लाह दिइन्छ। दर्ता पछि, तपाईंले अद्वितीय API कुञ्जी र ENDPOINT प्राप्त गर्न सक्नुहुन्छ।


# सेटअप 

यो नोटबुक चलाउन, तपाईंले आवश्यक पुस्तकालयहरू स्थापना गर्नैपर्नेछ `pip install -r requirements.txt` चलाएर।


In [ ]:
from semantic_kernel import __version__

__version__

तपाईंको सेमान्टिक कर्नेल संस्करण कम्तिमा १.२७.२ हुनुपर्छ।


तपाईंको .env फाइल सेटिङ र स्रोतहरू लोड गर्नुहोस् कृपया निश्चित गर्नुहोस् कि तपाईंले आफ्नो कुञ्जीहरू र सेटिङहरू थप्नुभएको छ र स्थानीय .env फाइल सिर्जना गर्नुभएको छ।


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure मा लगइन गर्नुहोस्

अब तपाईंले Azure मा लगइन गर्न आवश्यक छ। एउटा टर्मिनल खोल्नुहोस् र निम्न कमाण्ड चलाउनुहोस्:

```bash
az login
```

यस आदेशले तपाईंलाई तपाईंको Azure प्रमाण पत्रहरू प्रविष्ट गर्न अनुरोध गर्नेछ, जसले Azure AI Agent सेवा सही रूपमा काम गर्न सक्षम बनाउँछ।


# Explanation:
यो एउटा भेरिएबल हो जसले SERP (Search Engine Results Page) API सेवा पहुँच गर्नको लागि API कुञ्जी भण्डारण गर्छ। API कुञ्जी एउटा अनौठो परिचायक हो जुन तपाईंको खातासँग सम्बन्धित अनुरोधहरू प्रमाणीकरण गर्न प्रयोग गरिन्छ।

Purpose: यस लाइनको उद्देश्य API कुञ्जीलाई भेरिएबलमा भण्डारण गर्नु हो ताकि यसलाई SERP API सेवामा अनुरोधहरू प्रमाणीकरण गर्न प्रयोग गर्न सकियोस्। यो सेवा पहुँच गर्न र खोजहरू सञ्चालन गर्न API कुञ्जी आवश्यक हुन्छ।
How to Get a SERP API Key: SERP API कुञ्जी प्राप्त गर्न, यी सामान्य चरणहरू https://serpapi.com मा पछ्याउनुहोस् (ठ्याक्कै चरणहरू तपाईंले प्रयोग गरिरहेको विशिष्ट SERP API सेवामा निर्भर गरेर फरक हुन सक्छ):

Choose a SERP API Service: धेरै SERP API सेवाहरू उपलब्ध छन्, जस्तै SerpAPI, Google Custom Search JSON API, र अन्य। तपाईंका आवश्यकता अनुरूप सबैभन्दा उपयुक्त सेवा चयन गर्नुहोस्।

Sign Up for an Account: चयन गरिएको SERP API सेवाको वेबसाइटमा जानुहोस् र खाता खोल्न साइन अप गर्नुहोस्। केही आधारभूत जानकारी प्रदान गर्न र आफ्नो इमेल ठेगाना प्रमाणित गर्न आवश्यक पर्न सक्छ।

Create an API Key: साइन अप गरेपछि, आफ्नो खातामा लगइन गर्नुहोस् र API सेक्सन वा ड्यासबोर्डमा जानुहोस्। नयाँ API कुञ्जी सिर्जना वा उत्पादन गर्ने विकल्प खोज्नुहोस्।
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# व्याख्या:
BASE_URL: यो एउटा भेरिएबल हो जुन SERP API अन्तबिन्दुको लागि बेस URL भण्डारण गर्दछ। BASE_URL नामक भेरिएबल एउटा परम्परा हो जुन यो URL API अनुरोधहरू सुरु गर्ने बिन्दु भएको जनाउने लागि प्रयोग गरिन्छ।
'https://serpapi.com/search':

यो आधार URL स्ट्रिङ हो जुन BASE_URL भेरिएबललाई असाइन गरिएको छ। यसले SERP API प्रयोग गरेर खोजी क्वेरीहरू प्रदर्शन गर्ने अन्तबिन्दु प्रतिनिधित्व गर्दछ।

# उद्देश्य:
यस लाइनको उद्देश्य एक स्थिरांक परिभाषित गर्नु हो जुन SERP API को लागि बेस URL समेट्छ। यो URL API अनुरोधहरू निर्माण गर्नको लागि सुरु गर्ने बिन्दुको रूपमा प्रयोग गरिनेछ जुन खोजि सञ्चालनहरू प्रदर्शन गर्न प्रयोग हुन्छ।

# प्रयोग:
बेस URL लाई भेरिएबलमा परिभाषित गरेर, तपाईं यसलाई सजिलै आफ्नो कोडभर पुन: प्रयोग गर्न सक्नुहुन्छ जबसुकै तपाईंलाई SERP API मा अनुरोधहरू गर्नुपर्ने हुन्छ। यसले तपाईंको कोडलाई अधिक मर्मतयोग्य बनाउँछ र विभिन्न ठाउँहरूमा URL कडा रूपमा लेख्नु पर्ने त्रुटिहरूको जोखिम घटाउँछ। वर्तमान उदाहरण https://serpapi.com/search?engine=bing हो जुन Bing सर्च API प्रयोग गर्दैछ। विभिन्न API हरू https://Serpapi.com मा चयन गर्न सकिन्छ।


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# व्याख्या:

यहाँ तपाईंको प्लगइन कोड अवस्थित छ।

क्लास परिभाषा: `class BookingPlugin`: BookingPlugin नामक एक क्लास परिभाषित गर्दछ जसले होटल र उडान बुकिंगका लागि मेथडहरू समावेश गर्दछ।

होटल बुकिंग मेथड:

- `@kernel_function(description="booking hotel")`: होटल बुकिंगको लागि कर्नल फंक्शनको रूपमा यो फंक्शनलाई वर्णन गर्ने डेकोरेटर।
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: एनोटेटेड प्यारामिटरहरू र रिटर्न प्रकारसहित होटल बुकिंगको लागि मेथड परिभाषित गर्दछ।

यस मेथडले होटल बुकिंग अनुरोधको लागि प्यारामिटरहरूको डिक्सनरी तयार पार्छ र SERP API मा GET अनुरोध पठाउँछ। यसले प्रतिक्रिया स्थिति जाँच गर्छ र सफल भएमा होटल सम्पत्तिहरू फिर्ता गर्छ, वा अनुरोध विफल भएमा None फिर्ता गर्छ।

उडान बुकिंग मेथड:

- `@kernel_function(description="booking flight")`: उडान बुकिंगको लागि कर्नल फंक्शनको रूपमा यो फंक्शनलाई वर्णन गर्ने डेकोरेटर।
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: एनोटेटेड प्यारामिटरहरू र रिटर्न प्रकारसहित उडान बुकिंगको लागि मेथड परिभाषित गर्दछ।

यस मेथडले बाहिर निस्कने र फिर्ता उडान अनुरोधहरूको प्यारामिटरहरूको डिक्सनरी तयार पार्छ र SERP API मा GET अनुरोधहरू पठाउँछ। यसले प्रतिक्रिया स्थिति जाँच गर्छ र सफल भएमा उडान जानकारी परिणाम स्ट्रिङमा थप्छ, वा अनुरोध विफल भएमा त्रुटि सन्देश प्रिन्ट गर्छ। मेथडले उडान जानकारी समावेश गरिएको परिणाम स्ट्रिङ फिर्ता गर्छ।


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# व्याख्या:
आयात कथनहरू: Azure क्रेडेन्सियल, AI एजेण्ट, च्याट सन्देश सामग्री, लेखक भूमिका, र कर्नेल फङ्क्शन डेकोरेटरका लागि आवश्यक मोड्युलहरू आयात गर्नुहोस्।

असिन्क्रोनस सन्दर्भ प्रबन्धक: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): यसले Azure क्रेडेन्सियलहरू ह्यान्डल गर्न र AI एजेण्ट क्लाइन्ट सिर्जना गर्न असिन्क्रोनस सन्दर्भ प्रबन्धक सेट गर्दछ।

एजेण्ट नाम र निर्देशनहरू:
- `AGENT_NAME = "BookingAgent"`: एजेण्टको नाम परिभाषित गर्दछ।
- `AGENT_INSTRUCTIONS = """..."""`: बुकिङ अनुरोधहरू कसरी ह्यान्डल गर्ने सम्बन्धि एजेण्टलाई विस्तृत निर्देशनहरू प्रदान गर्दछ।

एजेण्ट परिभाषा सिर्जना गर्नुहोस्: `agent_definition = await client.agents.create_agent(...)`: निर्दिष्ट मोडेल, नाम, र निर्देशनहरूसँग एजेण्ट परिभाषा सिर्जना गर्दछ।

AzureAI एजेण्ट सिर्जना गर्नुहोस्: `agent = AzureAIAgent(...)`: क्लाइन्ट, एजेण्ट परिभाषा, र परिभाषित प्लगइन प्रयोग गरेर AzureAI एजेण्ट सिर्जना गर्दछ।

थ्रेड सिर्जना गर्नुहोस्: `thread: AzureAIAgentThread | None = None`: एजेण्टको लागि थ्रेड सिर्जना गर्नुहोस्। पहिलो पटक थ्रेड सिर्जना गर्नु आवश्यक छैन - यदि `None` को मान प्रदान गरिन्छ भने, पहिलो कलको क्रममा नयाँ थ्रेड सिर्जना हुनेछ र प्रतिक्रिया हिस्साको रूपमा फर्काइनेछ।

प्रयोगकर्ता इनपुटहरू: `user_inputs = ["..."]`: एजेण्टले प्रक्रिया गर्नका लागि प्रयोगकर्ता इनपुटहरूको सूची परिभाषित गर्दछ।

अन्तमा ब्लकमा, स्रोतहरू सफा गर्न थ्रेड र एजेण्ट मेटाउनुहोस्।


# प्रमाणीकरण

`DefaultAzureCredential` वर्ग Azure SDK for Python को भाग हो। यो Azure सेवाहरूमा प्रमाणीकरण गर्नको लागि डिफल्ट तरिका प्रदान गर्दछ। यो विशेष क्रममा विभिन्न विधिहरू प्रयोग गरी प्रमाणीकरण गर्ने प्रयास गर्दछ, जस्तै वातावरण परिवर्तनीयहरू, व्यवस्थापित पहिचान, र Azure CLI प्रमाणीकरणहरू।

असिंक्रोनस अपरेसनहरू: aio मोड्युलले संकेत गर्दछ कि DefaultAzureCredential वर्गले असिंक्रोनस अपरेसनहरू समर्थन गर्दछ। यसको मतलब तपाईं यसलाई asyncio सँग प्रयोग गरेर नन-ब्लकिङ प्रमाणीकरण अनुरोधहरू गर्न सक्नुहुन्छ।


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**छुटकारा सूचना**:  
यो कागजात AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धि हुन सक्छ। मूल कागजात यसको स्वदेशी भाषामा अधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा व्याख्यामा हामी जिम्मेवार रहेका छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
